In [11]:
from ARQMathCode.post_reader_record import DataReaderRecord
from question_answer.utils import examples_from_questions, dataloader_from_examples
from sentence_transformers import SentenceTransformer, losses

from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

In [13]:
device = "cuda"

clef_home_directory_file_path = '/home/xstefan3/arqmath/data/Collection'
dr = DataReaderRecord(clef_home_directory_file_path, limit_posts=100000)
# dr = DataReaderRecord(clef_home_directory_file_path)

reading users


In [3]:
from scipy.stats import zscore
from sentence_transformers.readers import InputExample
from sentence_transformers import SentencesDataset
from torch.utils.data import DataLoader
import numpy as np

In [4]:
all_examples = list(examples_from_questions(dr.post_parser.map_questions))
examples_len = len(all_examples)

/home/xstefan3/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:2419: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


Loading 20000
Loading 60000
Loading 110000
Loading 120000
Loading 140000
Loading 160000


In [5]:
train_dev_test_split = (int(0.8*examples_len), int(0.9*examples_len))
# train_dev_test_split = (160, 320)

In [6]:
model = SentenceTransformer('bert-base-wikipedia-sections-mean-tokens', logfile="train.log")

Logs go to file train.log


In [ ]:
train_loader = dataloader_from_examples(all_examples[:train_dev_test_split[0]], model,
                                        batch_size=16, shuffle=True)

Convert dataset:  25%|██▌       | 20304/79994 [01:46<04:43, 210.63it/s]

In [ ]:
dev_loader = dataloader_from_examples(all_examples[train_dev_test_split[0]:train_dev_test_split[1]], model,
                                      batch_size=16, shuffle=False)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model=model)
evaluator = EmbeddingSimilarityEvaluator(dev_loader, show_progress_bar=True, device=device)

In [ ]:
model.fit(train_objectives=[(train_loader, train_loss)],
          evaluator=evaluator,
          epochs=3,
          evaluation_steps=100,
          warmup_steps=10000,
          output_path="out",
          optimizer_params={'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False})